# Text classification from scratch

**Authors:** Mark Omernick, Francois Chollet<br>
**Date created:** 2019/11/06<br>
**Last modified:** 2020/05/17<br>
**Description:** Text sentiment classification starting from raw text files.

## Introduction

This example shows how to do text classification starting from raw text (as
a set of text files on disk). We demonstrate the workflow on the IMDB sentiment
classification dataset (unprocessed version). We use the `TextVectorization` layer for
 word splitting & indexing.

## Setup

In [2]:
import tensorflow as tf
import numpy as np
import os
from keras.utils import np_utils
from tensorflow.keras.layers import Embedding, Conv1D, AveragePooling1D, Bidirectional, LSTM, Dense

## Load the data: IMDB movie review sentiment classification

Let's download the data and inspect its structure.

In [3]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  10.0M      0  0:00:07  0:00:07 --:--:-- 17.7M


The `aclImdb` folder contains a `train` and `test` subfolder:

In [4]:
!ls aclImdb

imdbEr.txt  imdb.vocab	README	test  train


In [5]:
!ls aclImdb/test

labeledBow.feat  neg  pos  urls_neg.txt  urls_pos.txt


In [6]:
!ls aclImdb/train

labeledBow.feat  pos	unsupBow.feat  urls_pos.txt
neg		 unsup	urls_neg.txt   urls_unsup.txt


The `aclImdb/train/pos` and `aclImdb/train/neg` folders contain text files, each of
 which represents one review (either positive or negative):

In [7]:
!cat aclImdb/train/pos/6248_7.txt

Being an Austrian myself this has been a straight knock in my face. Fortunately I don't live nowhere near the place where this movie takes place but unfortunately it portrays everything that the rest of Austria hates about Viennese people (or people close to that region). And it is very easy to read that this is exactly the directors intention: to let your head sink into your hands and say "Oh my god, how can THAT be possible!". No, not with me, the (in my opinion) totally exaggerated uncensored swinger club scene is not necessary, I watch porn, sure, but in this context I was rather disgusted than put in the right context.<br /><br />This movie tells a story about how misled people who suffer from lack of education or bad company try to survive and live in a world of redundancy and boring horizons. A girl who is treated like a whore by her super-jealous boyfriend (and still keeps coming back), a female teacher who discovers her masochism by putting the life of her super-cruel "lover" 

We are only interested in the `pos` and `neg` subfolders, so let's delete the rest:

In [8]:
!rm -r aclImdb/train/unsup

In [9]:
for dir in range (10):
    os.mkdir(f'aclImdb/train/{dir}')

In [10]:
for dir in range (10):
    os.mkdir(f'aclImdb/test/{dir}')
!ls aclImdb/test 

0  2  4  6  8  labeledBow.feat	pos	      urls_pos.txt
1  3  5  7  9  neg		urls_neg.txt


You can use the utility `tf.keras.utils.text_dataset_from_directory` to
generate a labeled `tf.data.Dataset` object from a set of text files on disk filed
 into class-specific folders.

Let's use it to generate the training, validation, and test datasets. The validation
and training datasets are generated from two subsets of the `train` directory, with 20%
of samples going to the validation dataset and 80% going to the training dataset.

Having a validation dataset in addition to the test dataset is useful for tuning
hyperparameters, such as the model architecture, for which the test dataset should not
be used.

Before putting the model out into the real world however, it should be retrained using all
available training data (without creating a validation dataset), so its performance is maximized.

When using the `validation_split` & `subset` arguments, make sure to either specify a
random seed, or to pass `shuffle=False`, so that the validation & training splits you
get have no overlap.

In [11]:
directory = 'aclImdb/train'
 
for root, dirs, files in os.walk(directory):
    if root == 'aclImdb/train/neg' or root== 'aclImdb/train/pos':
        for file_ in files:
            if file_[-5]!='0':
                dir_ = int(file_[-5])-1
            else: 
                dir_ = int(file_[-6:-4])-1

            os.rename(''.join([root,'/',file_]), f'aclImdb/train/{dir_}/{file_}')

In [12]:
!rm -r aclImdb/train/neg
!rm -r aclImdb/train/pos

In [13]:
!ls aclImdb/train 

0  2  4  6  8  labeledBow.feat	urls_neg.txt  urls_unsup.txt
1  3  5  7  9  unsupBow.feat	urls_pos.txt


In [14]:
batch_size = 32
raw_train_ds = tf.keras.utils.text_dataset_from_directory(
    "aclImdb/train",
    batch_size=batch_size,
    validation_split=0.2,
    subset="training",
    seed=1337,
)

Found 25000 files belonging to 10 classes.
Using 20000 files for training.


In [15]:
directory = 'aclImdb/test'
 
for root, dirs, files in os.walk(directory):
    if root == 'aclImdb/test/neg' or root== 'aclImdb/test/pos':
        for file_ in files:
            if file_[-5]!='0':
                dir_ = int(file_[-5])-1
            else: 
                dir_ = int(file_[-6:-4])-1

            os.rename(''.join([root,'/',file_]), f'aclImdb/test/{dir_}/{file_}')

!rm -r aclImdb/test/neg
!rm -r aclImdb/test/pos

In [16]:
!ls aclImdb/test

0  1  2  3  4  5  6  7	8  9  labeledBow.feat  urls_neg.txt  urls_pos.txt


In [17]:
#dummy_train_labels = np_utils.to_categorical(train_labels)

In [18]:
batch_size = 16
raw_train_ds = tf.keras.utils.text_dataset_from_directory(
    "aclImdb/train",
    batch_size=batch_size,
    validation_split=0.2,
    subset="training",
    seed=1337,
)
raw_val_ds = tf.keras.utils.text_dataset_from_directory(
    "aclImdb/train",
    batch_size=batch_size,
    validation_split=0.2,
    subset="validation",
    seed=1337,
)
raw_test_ds = tf.keras.utils.text_dataset_from_directory(
    "aclImdb/test", batch_size=batch_size)

print(f"Number of batches in raw_train_ds: {raw_train_ds.cardinality()}")
print(f"Number of batches in raw_val_ds: {raw_val_ds.cardinality()}")
print(f"Number of batches in raw_test_ds: {raw_test_ds.cardinality()}")

Found 25000 files belonging to 10 classes.
Using 20000 files for training.
Found 25000 files belonging to 10 classes.
Using 5000 files for validation.
Found 25000 files belonging to 10 classes.
Number of batches in raw_train_ds: 1250
Number of batches in raw_val_ds: 313
Number of batches in raw_test_ds: 1563


In [19]:
for i, label in enumerate(raw_train_ds.class_names):
  print("Label", i, "corresponds to", label)

Label 0 corresponds to 0
Label 1 corresponds to 1
Label 2 corresponds to 2
Label 3 corresponds to 3
Label 4 corresponds to 4
Label 5 corresponds to 5
Label 6 corresponds to 6
Label 7 corresponds to 7
Label 8 corresponds to 8
Label 9 corresponds to 9


Let's preview a few samples:

In [20]:
for comments, labels in raw_train_ds.take(1):
    for i in range(5):
        print(comments.numpy()[i])
        print(labels.numpy()[i])

b"I think that Elisabeth Rohm, though she may try hard, is not very good at all. I guess it was because of budget that she may have been the only one they could get for that price. I mainly watched it for the performance of Myron Natwick, whose work I know very well.<br /><br />He was the most believable and without doubt the most compelling to watch. When he wasn't on the screen, the thing went dead. This was filmed in Vancouver. He gave me the creeps, but be assured that in real life he is a kind, funny compassionate man. He even said playing that role gave him the creeps.<br /><br />I'll watch anything with him in it, but Elisabeth Rohm - never again. She was as exciting as lint on Law and Order. Maybe a very nice person, but no actress."
1
b'Boris Karloff is Matthias Morteval, a dying, lonely old nut who lives in Morhenge Mansion with some servants and tells his doctor friend, "Don\'t try to doctor me, doctor! I\'m disgustingly healthy!" He invites his nieces and nephews to his hom

## Prepare the data

In particular, we remove `<br />` tags.

In [21]:
from tensorflow.keras.layers import TextVectorization
import string
import re

# Having looked at our data above, we see that the raw text contains HTML break
# tags of the form '<br />'. These tags will not be removed by the default
# standardizer (which doesn't strip HTML). Because of this, we will need to
# create a custom standardization function.
def custom_standardization(input_data):
    lowercase = tf.strings.lower(input_data)
    stripped_html = tf.strings.regex_replace(lowercase, "<br />", " ")
    return tf.strings.regex_replace(
        stripped_html, f"[{re.escape(string.punctuation)}]", ""
    )


# Model constants.
max_features = 20000
embedding_dim = 128
sequence_length = 500

# Now that we have our custom standardization, we can instantiate our text
# vectorization layer. We are using this layer to normalize, split, and map
# strings to integers, so we set our 'output_mode' to 'int'.
# Note that we're using the default split function,
# and the custom standardization defined above.
# We also set an explicit maximum sequence length, since the CNNs later in our
# model won't support ragged sequences.
vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=max_features,
    output_mode="int",
    output_sequence_length=sequence_length,
)

# Now that the vocab layer has been created, call `adapt` on a text-only
# dataset to create the vocabulary. You don't have to batch, but for very large
# datasets this means you're not keeping spare copies of the dataset in memory.

# Let's make a text-only dataset (no labels):
text_ds = raw_train_ds.map(lambda x, y: x)
# Let's call `adapt`:
vectorize_layer.adapt(text_ds)

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


## Two options to vectorize the data

There are 2 ways we can use our text vectorization layer:

**Option 1: Make it part of the model**, so as to obtain a model that processes raw
 strings, like this:

```python
text_input = tf.keras.Input(shape=(1,), dtype=tf.string, name='text')
x = vectorize_layer(text_input)
x = layers.Embedding(max_features + 1, embedding_dim)(x)
...
```

**Option 2: Apply it to the text dataset** to obtain a dataset of word indices, then
 feed it into a model that expects integer sequences as inputs.

An important difference between the two is that option 2 enables you to do
**asynchronous CPU processing and buffering** of your data when training on GPU.
So if you're training the model on GPU, you probably want to go with this option to get
 the best performance. This is what we will do below.

If we were to export our model to production, we'd ship a model that accepts raw
strings as input, like in the code snippet for option 1 above. This can be done after
 training. We do this in the last section.


In [22]:

def vectorize_text(text, label):
    text = tf.expand_dims(text, -1)
    return vectorize_layer(text), label


# Vectorize the data.
train_ds = raw_train_ds.map(vectorize_text)
val_ds = raw_val_ds.map(vectorize_text)
test_ds = raw_test_ds.map(vectorize_text)

# Do async prefetching / buffering of the data for best performance on GPU.
train_ds = train_ds.cache().prefetch(buffer_size=10)
val_ds = val_ds.cache().prefetch(buffer_size=10)
test_ds = test_ds.cache().prefetch(buffer_size=10)

## Build a model

We choose a simple 1D convnet starting with an `Embedding` layer.

In [23]:
from tensorflow.keras import layers

In [24]:
# A integer input for vocab indices.
inputs = tf.keras.Input(shape=(None,), dtype="int64")

# Next, we add a layer to map those vocab indices into a space of dimensionality
# 'embedding_dim'.
x = layers.Embedding(max_features, embedding_dim)(inputs)
#x = layers.Dropout(0.5)(x)

# Conv1D + global max pooling
# x = layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3)(x)
# x = layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3)(x)
x = layers.LSTM(128, return_sequences=True, return_state=False)(x)
x = layers.LSTM(256, return_sequences=True, return_state=False)(x)
x = layers.GlobalMaxPooling1D()(x)

# We add a vanilla hidden layer:
x = layers.Dense(128, activation="relu")(x)
#x = layers.Dropout(0.5)(x)

# We project onto a single unit output layer, and squash it with a sigmoid:
predictions = tf.keras.layers.Dense(10, activation="softmax", name="predictions")(x)

model = tf.keras.Model(inputs, predictions)

# Compile the model with binary crossentropy loss and an adam optimizer.
model.compile(loss='sparse_categorical_crossentropy', optimizer="adam", metrics=["accuracy"])

In [25]:
# A integer input for vocab indices.
inputs = tf.keras.Input(shape=(None,), dtype="int64")

# Next, we add a layer to map those vocab indices into a space of dimensionality
# 'embedding_dim'.
x = layers.Embedding(max_features, embedding_dim)(inputs)
x = layers.Conv1D(filters = 32, kernel_size = 3, padding = 'same', activation = 'relu')(x)
x = layers.AveragePooling1D(pool_size = 2)(x)
x = layers.Bidirectional(LSTM(200, dropout = 0.5))(x)
# We project onto a single unit output layer, and squash it with a sigmoid:
predictions = tf.keras.layers.Dense(10, activation="softmax", name="predictions")(x)

model = tf.keras.Model(inputs, predictions)

# Compile the model with binary crossentropy loss and an adam optimizer.
model.compile(loss='sparse_categorical_crossentropy', optimizer="adam", metrics=["accuracy"])

In [26]:
inputs = tf.keras.Input(shape=(None,), dtype="int64")
###x = layers.Embedding(129892, 32)(inputs)
x = layers.Embedding(max_features, embedding_dim)(inputs)
x = layers.Bidirectional(LSTM(75, dropout = 0.1))(x)
predictions = tf.keras.layers.Dense(10, activation="softmax", name="predictions")(x)

model = tf.keras.Model(inputs, predictions)

# Compile the model with binary crossentropy loss and an adam optimizer.
model.compile(loss='sparse_categorical_crossentropy', optimizer="adam", metrics=["accuracy"])

# self.embed = T.nn.Embedding(129892, 32)
# self.lstm = T.nn.LSTM(32, 75)
# self.drop = T.nn.Dropout(0.10)
# self.fc1 = T.nn.Linear(75, 10)  
# self.fc2 = T.nn.Linear(10, 2)  # 0=neg, 1=pos

# def forward(self, x):
# # x = review/sentence. length = 50 (fixed w/ padding)
# z = self.embed(x) 
# z = z.view(50, 1, 32)  # "seq batch input"
# lstm_oupt, (h_n, c_n) = self.lstm(z)
# z = lstm_oupt[-1]
# z = self.drop(z)
# z = T.tanh(self.fc1(z)) 
# z = self.fc2(z)  # CrossEntropyLoss will apply softmax
# return z  

In [31]:
inputs = tf.keras.Input(shape=(None,), dtype="int64")

x = layers.Embedding(max_features, embedding_dim)(inputs)
x = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(x)
x = layers.Bidirectional(layers.LSTM(64))(x)
predictions = tf.keras.layers.Dense(10, activation="softmax", name="predictions")(x)

model = tf.keras.Model(inputs, predictions)

# Compile the model with binary crossentropy loss and an adam optimizer.
model.compile(loss='sparse_categorical_crossentropy', optimizer="adam", metrics=["accuracy"])


## Train the model

In [32]:
epochs = 10

# Fit the model using the train and test datasets.
model.fit(train_ds, validation_data=val_ds, epochs=epochs)

Epoch 1/10
1250/1250 [==============================] - 124s 94ms/step - loss: 1.8418 - accuracy: 0.3067 - val_loss: 1.7912 - val_accuracy: 0.3298
Epoch 2/10
1250/1250 [==============================] - 64s 51ms/step - loss: 1.6300 - accuracy: 0.3852 - val_loss: 1.6618 - val_accuracy: 0.3634
Epoch 3/10
1250/1250 [==============================] - 63s 50ms/step - loss: 1.4508 - accuracy: 0.4391 - val_loss: 1.6665 - val_accuracy: 0.3644
Epoch 4/10
1250/1250 [==============================] - 65s 52ms/step - loss: 1.3044 - accuracy: 0.4942 - val_loss: 1.6619 - val_accuracy: 0.3678
Epoch 5/10
1250/1250 [==============================] - 65s 52ms/step - loss: 1.1938 - accuracy: 0.5315 - val_loss: 1.7194 - val_accuracy: 0.3802
Epoch 6/10
1250/1250 [==============================] - 64s 51ms/step - loss: 1.0724 - accuracy: 0.5755 - val_loss: 1.8287 - val_accuracy: 0.3774
Epoch 7/10
1250/1250 [==============================] - 65s 52ms/step - loss: 0.9369 - accuracy: 0.6384 - val_loss: 2.0353 

## Evaluate the model on the test set

In [28]:
model.evaluate(test_ds)

1563/1563 [==============================] - 21s 13ms/step - loss: 3.3232 - accuracy: 0.3380


[3.3231983184814453, 0.33803999423980713]

## Make an end-to-end model

If you want to obtain a model capable of processing raw strings, you can simply
create a new model (using the weights we just trained):

In [30]:
# A string input
inputs = tf.keras.Input(shape=(1,), dtype="string")
# Turn strings into vocab indices
indices = vectorize_layer(inputs)
# Turn vocab indices into predictions
outputs = model(indices)

# Our end to end model
end_to_end_model = tf.keras.Model(inputs, outputs)
end_to_end_model.compile(
    loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"]
)

# Test it with `raw_test_ds`, which yields raw strings
end_to_end_model.evaluate(raw_test_ds)

ValueError: ignored

In [ ]:
yhat_ = model.predict(test_ds)

In [ ]:
yhat_